# Training a hadronization net

In [1]:
import random as rnd
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Concatenate, Lambda
from tensorflow.keras.layers import Layer, Flatten
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import OneHotEncoder

In [2]:
# data from HadronizationPrep.ipynb
x_raw = np.loadtxt('x_raw.dat')
y_raw = np.loadtxt('y_raw.dat')

In [3]:
# postprocess (norm everything to E1)
for i, x in enumerate(x_raw):
    E1 = x_raw[i,0]
    E2 = x_raw[i,4]
    x_raw[i,:8] = x_raw[i,:8]/(E1+E2)
    y_raw[i,1:4] = y_raw[i,1:4]/(E1+E2)

In [4]:
y_raw

array([[ 1.00000000e+00,  1.23860298e-04, -3.87150854e-03,
        -8.00250497e-01,  2.11000000e+02],
       [ 1.00000000e+00, -1.69576004e-02,  2.18929668e-02,
        -9.99483254e-01,  3.11000000e+02],
       [-1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.00000000e+00, -1.19333048e-04,  1.10517411e-04,
         5.31168608e-01,  2.23000000e+02],
       [ 1.00000000e+00,  1.17364570e-03,  5.09638066e-05,
         8.85231046e-01,  2.21200000e+03],
       [ 1.00000000e+00, -4.01312924e-04,  2.28374011e-03,
         9.99996333e-01,  2.11000000e+02]])

# Momentum model definition

In [41]:
# this is a permanent dropout layer designed to add a lot of randomness
class PermaDropout(Layer):
    def __init__(self, rate):
        super(PermaDropout, self).__init__()
        self.rate = rate

    def call(self, inputs):
        return tf.nn.dropout(inputs, rate=self.rate)

# this is the definition of our custom loss function
REG_LQCD = 1.0 
REG_LQCD_SQ = REG_LQCD**2
REG_TENS = K.constant(REG_LQCD_SQ,shape=3)
REG_TENS4 = K.constant(REG_LQCD_SQ,shape=4)
BATCH_SIZE = 256
REG_TENSB4 = K.constant(REG_LQCD_SQ,shape=[BATCH_SIZE,4])

# Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
# note: definition is asymmetric in penalizing large momenta
def lossx(y_true,y_pred):
    #return 1.-y_true[0]*K.tanh(y_pred[0]) + 0.5*(1.+y_true[0])*K.mean(K.square(y_pred[1:] - y_true[1:])/(K.square(y_true[1:])+REG_TENS), axis=-1)
    return 1.-y_true[0]*K.tanh(y_pred[0]) + 0.5*(1.+y_true[0])*K.mean([0,1,1,1]*K.square(y_pred - y_true)/(K.square(y_true)+REG_TENS4), axis=-1)
        
        
def cust_lossx():    
    def lossx(y_true,y_pred):
        A = y_true[:,0]
        return 1.-A*K.tanh(y_pred[:,0]) + 0.5*(1.+A)*K.mean([0,1,1,1]*K.square(y_pred - y_true)/(K.square(y_true)+REG_TENSB4), axis=-1)

    return lossx

# use cross product
def epsijkchoose(i,j,k):
    if i == j or j == k or k == i:
        return 0
    if i == 0:
        if j == 1:
            return 1
        if j == 2:
            return -1
    if i == 1:
        if j == 2:
            return 1
        if j == 0:
            return -1
    if i == 2:
        if j == 0:
            return 1
        if j == 1:
            return -1
epsijk = np.array([ [ [ epsijkchoose(i,j,k) for k in range(3)] for j in range(3)] for i in range(3)]).astype(np.float32)

def cross(x,y):
    x = x[:,-3:]
    y = y[:,-3:]
    epsijkbatch = K.expand_dims(EPS_IJK,len(x),axis=0)
    epsijkbatch = K.batch_dot(epsijkbatch, x, axes=(2, 1))
    epsijkbatch = K.batch_dot(epsijkbatch, y, axes=(2, 1))
    return epsijkbatch

def normcross(x,y):
    return K.sqrt(K.sum(K.square(cross(x,y))))

def normcross(x,y):
    lx = x[:,2]*y[:,3] - x[:,3]*y[:,2]
    ly = x[:,3]*y[:,1] - x[:,1]*y[:,3]
    lz = x[:,1]*y[:,2] - x[:,2]*y[:,1]
    lsq = np.array(lx*lx + ly*ly + lz*lz)
    return lsq**0.5

def lprod(x,y,i,j):
    return K.batch_dot(x[:,i:i+1],y[:,j:j+1]) - K.batch_dot(x[:,j:j+1],y[:,i:i+1])

def normcross(x,y):
    lx = lprod(x,y,2,3)
    ly = lprod(x,y,3,1)
    lz = lprod(x,y,1,2)
    lsq = K.sqrt(K.square(lx) + K.square(ly) + K.square(lz))
    return K.flatten(lsq)

def rtdot(x,y):
    xdysqrt = K.sqrt(K.batch_dot(y[:,1:4]-x[:,1:4],y[:,1:4]))
    return K.flatten(xdysqrt)

def normdot(x,y):
    xdy = K.batch_dot(y[:,1:4]-x[:,1:4],y[:,1:4])
    return K.flatten(xdy)

def lossx(y_true,y_pred):
    A = y_true[:,0]
    B = 3*A - 2
    yabs = K.sqrt(K.mean([0,1,1,1]*K.square(y_true), axis=-1)+1e-14)
    return 1.-B*K.tanh(y_pred[:,0]) + 0.5*(1.+A)/yabs*(K.mean([0,1,1,1]*K.square(y_pred - y_true), axis=-1) + normdot(y_pred, y_true) + normcross(y_pred, y_true))


def cust_lossx():    
    def lossx(y_true,y_pred):
        A = y_true[:,0]
        if A == -1:
            A = -10
        return 1.-A*K.tanh(y_pred[:,0]) + 0.5*(1.+A)*(K.mean([0,1,1,1]*K.square(y_pred - y_true), axis=-1) + normcross(y_pred, y_true))

    return lossx

In [42]:
print(normdot(np.array([[1,0.0005,0.0005,0.5]]),np.array([[1,0.0005,0.0005,-0.5]])))
print(normcross(np.array([[1,0.005,0.005,0.5]]),np.array([[1,0.0005,0.005,-0.5]])))
lossx(np.array([[-1,0.005,0.005,0.5]]),np.array([[100,0.0005,0.005,-0.5]]))

tf.Tensor([0.5], shape=(1,), dtype=float64)
tf.Tensor([0.0057064], shape=(1,), dtype=float64)


<tf.Tensor: id=555644, shape=(1,), dtype=float64, numpy=array([6.])>

In [7]:
TRAINCUT = int((int(0.85*len(x_raw))//BATCH_SIZE) * BATCH_SIZE)
VALCUT = int((int(0.1*len(x_raw))//BATCH_SIZE) * BATCH_SIZE)
TESTCUT = int((int(0.05*len(x_raw))//BATCH_SIZE) * BATCH_SIZE)
print("{} {} {}".format(TRAINCUT,VALCUT,TESTCUT))

# Note with this data, I am assuming it doesn't need to be shuffled, but may wish to do that later

x_train = x_raw[:TRAINCUT,:8]
y_train = y_raw[:TRAINCUT,:4]

x_val = x_raw[TRAINCUT:TRAINCUT+VALCUT,:8]
y_val = y_raw[TRAINCUT:TRAINCUT+VALCUT,:4]

x_test = x_raw[-TESTCUT:,:8]
y_test = y_raw[-TESTCUT:,:4]

513792 60416 30208


# Momentum VAE

In [51]:
class VarAutoEncoder():
    # feature extractor model
    def __init__(self):
        self.z_part12 = 8 
        self.z_part3 = 4 
        self.z_dim = 3 
        self._build()
    
    def _build(self):
        part12_input = Input(shape=(self.z_part12,), name='part12_input')
        part3_input = Input(shape=(self.z_part3,), name='part3_input')
        
        # encoder
        enc_cat = Concatenate()([part3_input, part12_input])
        enc = Dense(32, activation='relu')(enc_cat)
        enc = Dense(32, activation='relu')(enc)
        enc = Dense(32, activation='relu')(enc)
        # construct variational aspect
        self.mu = Dense(self.z_dim, name='mu')(enc)
        self.log_var = Dense(self.z_dim, name='log_var')(enc)
        #enc_outprep = Concatenate()([self.mu, self.log_var])
        #self.encoder_mu_log_var = Model([part3_input,part12_input], [self.mu, self.log_var])
        
        def sampling(args):
            mu, log_var = args
            epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
            return mu + K.exp(log_var / 2) * epsilon

        enc_output = Lambda(sampling, name='encoder_output')([self.mu, self.log_var])
        self.encoder = Model([part3_input,part12_input],enc_output)
        
        # decoder
        encoded_input = Input(shape=(self.z_dim,), name='enc_part3_input')
        dec_input = Concatenate()([encoded_input, part12_input])
        dec = Dense(32, activation='relu')(dec_input)
        #PermaDropout(0.5),
        dec = Dense(32, activation='relu')(dec)
        #PermaDropout(0.5),
        dec = Dense(32, activation='relu')(dec)
        #PermaDropout(0.5),
        dec_output = Dense(self.z_part3, activation='linear')(dec)
        self.decoder = Model([encoded_input, part12_input], dec_output)
        
        # the autoencoder
        model_input = [part3_input,part12_input]
        model_output = self.decoder([enc_output,part12_input])

        self.autoencoder = Model(model_input, model_output)
    
    def compile(self, learning_rate):
        self.learning_rate = learning_rate
        optimizer = Adam(lr=learning_rate)
        
        def vae_kl_loss(y_true, y_pred):
            kl_loss =  -0.5 * K.sum(1 + self.log_var - K.square(self.mu) - K.exp(self.log_var), axis = 1)
            return kl_loss
        
        def lossx(y_true,y_pred, emitScale = 1.5):
            A = y_true[:,0]
            """ Note: This scale encourages the program to get the emission prob correct.  
                If too small (e.g. 1), it is never favorable.  If too large, e.g. 5 it is always favorable,
                but at the cost of the other part of the loss.
            """
            B = emitScale*A - (emitScale-1)
            yabs = K.sqrt(K.mean([0,1,1,1]*K.square(y_true), axis=-1)+1e-14)
            return 1.-B*K.tanh(y_pred[:,0]) + 0.5*(1.+A)/yabs*(K.mean([0,1,1,1]*K.square(y_pred - y_true), axis=-1) + normdot(y_pred, y_true) + normcross(y_pred, y_true))
        
        def vae_loss(y_true, y_pred, scale = 1):
            loss_x = lossx(y_true, y_pred)
            kl_loss = vae_kl_loss(y_true, y_pred)
            return  scale*loss_x + kl_loss
        
        """ Here, I need to set experimental_run_tf_function=False in order for this to execute.
            I do not know why exactly, but with a TF2 upgrade this may go away. See discussion:
            https://github.com/tensorflow/probability/issues/519
        """
        self.autoencoder.compile(optimizer=optimizer, loss = vae_loss, metrics = [vae_kl_loss, lossx],
                                experimental_run_tf_function=False)
        
    def train(self, x_train, y_train, batch_size, epochs, validation_data=None):
        
        if validation_data is not None:
            x_val = validation_data[0]
            y_val = validation_data[1]
            validation_data=[[y_val,x_val],y_val]

        self.autoencoder.fit([y_train, x_train], 
                             y_train,
                             batch_size = batch_size,
                             shuffle = True,
                             epochs = epochs,
                             validation_data=validation_data)

In [52]:
var_auto_encoder = VarAutoEncoder()

In [53]:
LEARNING_RATE = 0.001
var_auto_encoder.compile(LEARNING_RATE)

In [ ]:
var_auto_encoder.train(x_train, y_train, 128, 15, validation_data=[x_val,y_val])

Train on 513792 samples, validate on 60416 samples
Epoch 1/15
513792/513792 [==============================] - 40s 79us/sample - loss: 0.9568 - vae_kl_loss: 0.0490 - lossx: 0.9078 - val_loss: 0.6640 - val_vae_kl_loss: 0.0882 - val_lossx: 0.5758
Epoch 2/15
513792/513792 [==============================] - 35s 69us/sample - loss: 0.6484 - vae_kl_loss: 0.0978 - lossx: 0.5505 - val_loss: 0.6115 - val_vae_kl_loss: 0.0929 - val_lossx: 0.5186
Epoch 3/15
449152/513792 [=========================>....] - ETA: 4s - loss: 0.5931 - vae_kl_loss: 0.1025 - lossx: 0.4906

In [21]:
idx = 31
xs = x_train[idx:idx+1]
ys = y_train[idx:idx+1]
enc_out = var_auto_encoder.encoder.predict([ys,xs])
dec_out = var_auto_encoder.decoder.predict([enc_out,xs])
print(xs)
print(ys)
print(enc_out)
print(dec_out)
print(ys)

[[ 0.73546911  0.04295983  0.01637338  0.7340304   0.26453089 -0.04474383
   0.14407936  0.21729179]]
[[-1.  0.  0.  0.]]
[[-0.4313089   0.34734607  2.5874135 ]]
[[2.8373638e+01 2.5901036e-02 8.8629372e-02 1.1157228e+00]]
[[-1.  0.  0.  0.]]


# Momentum AE

In [91]:
class AutoEncoder():
    # feature extractor model
    def __init__(self):
        self.z_part12 = 8 
        self.z_part3 = 4 
        self.z_dim = 3 
        self._build()
    
    def _build(self):
        part12_input = Input(shape=(self.z_part12,), name='part12_input')
        part3_input = Input(shape=(self.z_part3,), name='part3_input')
        
        # encoder
        enc_cat = Concatenate()([part3_input, part12_input])
        enc = Dense(32, activation='relu')(enc_cat)
        enc = Dense(32, activation='relu')(enc)
        enc = Dense(32, activation='relu')(enc)
        encoded = Dense(self.z_dim, activation='linear')(enc)
        self.encoder = Model([part3_input,part12_input],encoded)
        
        # decoder
        encoded_input = Input(shape=(self.z_dim,), name='enc_part3_input')
        dec_input = Concatenate()([encoded_input, part12_input])
        dec = Dense(64, activation='relu')(dec_input)
        #PermaDropout(0.5),
        dec = Dense(64, activation='relu')(dec)
        #PermaDropout(0.5),
        dec = Dense(64, activation='relu')(dec)
        #PermaDropout(0.5),
        dec_output = Dense(self.z_part3, activation='linear')(dec)
        self.decoder = Model([encoded_input, part12_input], dec_output)
        
        # the autoencoder
        model_input = [part3_input,part12_input]
        model_output = self.decoder([encoded,part12_input])

        self.autoencoder = Model(model_input, model_output)
    
    def compile(self, learning_rate):
        self.learning_rate = learning_rate
        optimizer = Adam(lr=learning_rate)
        
        def lossx(y_true,y_pred):
            A = y_true[:,0]
            yabs = K.sqrt(K.mean([0,1,1,1]*K.square(y_true), axis=-1)+1e-14)
            return 1.-A*K.tanh(y_pred[:,0]) + 0.5*(1.+A)/yabs*(K.mean([0,1,1,1]*K.square(y_pred - y_true), axis=-1) + normdot(y_pred, y_true) + normcross(y_pred, y_true))
        
        self.autoencoder.compile(optimizer=optimizer, loss = lossx)
        
    def train(self, x_train, y_train, batch_size, epochs, validation_data=None):
        
        if validation_data is not None:
            x_val = validation_data[0]
            y_val = validation_data[1]
            validation_data=[[y_val,x_val],y_val]

        self.autoencoder.fit([y_train, x_train], 
                             y_train,
                             batch_size = batch_size,
                             shuffle = True,
                             epochs = epochs,
                             validation_data=validation_data
                            
        )

In [92]:
auto_encoder = AutoEncoder()

In [93]:
LEARNING_RATE = 0.001
auto_encoder.compile(LEARNING_RATE)

In [94]:
auto_encoder.train(x_train, y_train, 128, 15, validation_data=[x_val,y_val])

Train on 513792 samples, validate on 60416 samples
Epoch 1/15
513792/513792 [==============================] - 30s 59us/sample - loss: 1.0019 - val_loss: 0.7041
Epoch 2/15
513792/513792 [==============================] - 32s 62us/sample - loss: 0.5596 - val_loss: 0.3340
Epoch 3/15
513792/513792 [==============================] - 30s 59us/sample - loss: 0.4461 - val_loss: 0.3782
Epoch 4/15
513792/513792 [==============================] - 30s 58us/sample - loss: 0.3584 - val_loss: 0.8633
Epoch 5/15
513792/513792 [==============================] - 30s 58us/sample - loss: 0.3065 - val_loss: 0.4606
Epoch 6/15
513792/513792 [==============================] - 30s 58us/sample - loss: 0.2541 - val_loss: 0.2671
Epoch 7/15
513792/513792 [==============================] - 30s 59us/sample - loss: 0.2281 - val_loss: 0.2101
Epoch 8/15
513792/513792 [==============================] - 30s 58us/sample - loss: 0.2937 - val_loss: 0.3135
Epoch 9/15
513792/513792 [==============================] - 30s 59us/

In [104]:
idx = 22
xs = x_train[idx:idx+1]
ys = y_train[idx:idx+1]
enc_out = auto_encoder.encoder.predict([ys,xs])
dec_out = auto_encoder.decoder.predict([enc_out,xs])
print(xs)
print(ys)
print(enc_out)
print(dec_out)
print(ys)

[[ 0.59124529  0.00103248  0.00326767  0.59123535  0.40875471 -0.01086593
  -0.00165995  0.40860688]]
[[1.00000000e+00 6.57285212e-04 4.81170271e-03 4.01718718e-01]]
[[30.588957  -1.5729476  2.5272653]]
[[3.6106445e+01 1.2611160e-03 5.5991746e-03 4.3408090e-01]]
[[1.00000000e+00 6.57285212e-04 4.81170271e-03 4.01718718e-01]]


# Initial momentum model

In [132]:
def define_momentum_model():
    # feature extractor model
    model = Sequential([
        Dense(64, activation='relu',input_shape=[8]),
        PermaDropout(0.5),
        Dense(64, activation='relu'),
        PermaDropout(0.5),
        Dense(4, activation='linear')
        ])
    print(model.summary())
    # Compile the model
    model.compile(optimizer='adam',loss=cust_lossx())
    return model

In [133]:
model = define_momentum_model()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 64)                576       
_________________________________________________________________
perma_dropout_18 (PermaDropo (None, 64)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                4160      
_________________________________________________________________
perma_dropout_19 (PermaDropo (None, 64)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 4)                 260       
Total params: 4,996
Trainable params: 4,996
Non-trainable params: 0
_________________________________________________________________
None


In [134]:
model.optimizer.lr = 0.001

## Training momentum model

In [136]:
history = model.fit(x_train, y_train, epochs=30, batch_size = BATCH_SIZE, validation_data=[x_val,y_val])

Train on 513792 samples, validate on 60416 samples
Epoch 1/30
513792/513792 [==============================] - 24s 48us/sample - loss: 0.3210 - val_loss: 0.3052
Epoch 2/30
513792/513792 [==============================] - 21s 42us/sample - loss: 0.3029 - val_loss: 0.3034
Epoch 3/30
513792/513792 [==============================] - 22s 43us/sample - loss: 0.3019 - val_loss: 0.3030
Epoch 4/30
513792/513792 [==============================] - 23s 44us/sample - loss: 0.3015 - val_loss: 0.3024
Epoch 5/30
513792/513792 [==============================] - 21s 42us/sample - loss: 0.3012 - val_loss: 0.3023
Epoch 6/30
 17152/513792 [>.............................] - ETA: 20s - loss: 0.2986

KeyboardInterrupt: 

In [145]:
idx = 202
print(x_train[idx])
print(y_train[idx])
print(model.predict(x_train[idx:idx+1]))

[ 0.18513115 -0.00340915  0.00204126  0.18502525  0.81486885  0.00235271
  0.00728108 -0.81481431]
[ 1.00000000e+00 -7.78584587e-04  3.80717898e-03  1.37925427e-01]
[[6.6870565e+00 6.9317431e-04 1.5475576e-04 1.5962045e-03]]


In [144]:
idx = 102
print(x_train[idx])
print(y_train[idx])
print(model.predict(x_train[idx:idx+1]))

[ 5.46594976e-01 -1.42339427e-01  4.69652409e-03  5.27704385e-01
  4.53405024e-01 -3.32098278e-01  5.41031581e-04 -3.08685196e-01]
[-1.  0.  0.  0.]
[[9.3255873e+00 8.0299063e-04 1.2794735e-03 6.3834339e-02]]


# Flavor model definition

In [37]:
TRAINCUT = int((int(0.85*len(x_raw))//BATCH_SIZE) * BATCH_SIZE)
VALCUT = int((int(0.1*len(x_raw))//BATCH_SIZE) * BATCH_SIZE)
TESTCUT = int((int(0.05*len(x_raw))//BATCH_SIZE) * BATCH_SIZE)
print("{} {} {}".format(TRAINCUT,VALCUT,TESTCUT))

# Note with this data, I am assuming it doesn't need to be shuffled, but may wish to do that later

encin = OneHotEncoder(sparse = False)
xf_ohe = encin.fit_transform(x_raw[:,8:].astype(int))

encout = OneHotEncoder(sparse = False)
yf_ohe = encout.fit_transform(y_raw[:,4:].astype(int))

xf_train = xf_ohe[:TRAINCUT]
yf_train = yf_ohe[:TRAINCUT]

xf_val = xf_ohe[TRAINCUT:TRAINCUT+VALCUT]
yf_val = yf_ohe[TRAINCUT:TRAINCUT+VALCUT]

xf_test = xf_ohe[-TESTCUT:]
yf_test = yf_ohe[-TESTCUT:]

513920 60416 30208


In [46]:
nIn = xf_ohe.shape[1]

In [38]:
yf_train.shape

(513920, 89)

In [39]:
inflavorset = [ 21, 1,-1,2,-2,3,-3 ]
outflavorset = [ ]
for x in x_raw[:,8:]:
    for y in x:
        if y not in inflavorset:
            inflavorset.append(y)

for y in y_raw[:,4]:
    if y not in outflavorset:
        outflavorset.append(y)

nInStates = len(inflavorset)
nOutStates = len(outflavorset)

In [47]:
def define_flavor_model():
    # feature extractor model
    inputs = Input(shape=(nIn,))
    l1 = Dense(64, activation='relu')(inputs)
    l2 = Dense(64, activation='relu')(l1)
    outputs = Dense(nOutStates, activation='softmax')(l2)
    model = Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    # Compile the model
    model.compile(optimizer='adam',loss='categorical_crossentropy')
    return model

In [48]:
flavormodel=define_flavor_model()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 39)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                2560      
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 89)                5785      
Total params: 12,505
Trainable params: 12,505
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
history = flavormodel.fit(xf_train, yf_train, epochs=30, batch_size = BATCH_SIZE, validation_data=[xf_val,yf_val])

Train on 513920 samples, validate on 60416 samples
Epoch 1/30
513920/513920 [==============================] - 29s 56us/sample - loss: 2.2018 - val_loss: 2.1454
Epoch 2/30
513920/513920 [==============================] - 30s 58us/sample - loss: 2.1457 - val_loss: 2.1407
Epoch 3/30
513920/513920 [==============================] - 36s 70us/sample - loss: 2.1427 - val_loss: 2.1407
Epoch 4/30
513920/513920 [==============================] - 32s 63us/sample - loss: 2.1409 - val_loss: 2.1375
Epoch 5/30
513920/513920 [==============================] - 32s 62us/sample - loss: 2.1392 - val_loss: 2.1378
Epoch 6/30
513920/513920 [==============================] - 31s 60us/sample - loss: 2.1378 - val_loss: 2.1378
Epoch 7/30
513920/513920 [==============================] - 28s 55us/sample - loss: 2.1368 - val_loss: 2.1365
Epoch 8/30
513920/513920 [==============================] - 32s 61us/sample - loss: 2.1363 - val_loss: 2.1366
Epoch 9/30
513920/513920 [==============================] - 32s 62us/

In [51]:
yf_train.shape

(513920, 89)